In [2]:
import os

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from typing import Dict

In [3]:
documents = []

In [4]:
loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/01_프로젝트 계획서.pdf")
pdf_documents = loader.load()

In [5]:
documents.extend(pdf_documents)

In [6]:
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk = chunk_splitter.split_documents(documents)

In [7]:
embeddings = HuggingFaceBgeEmbeddings()

c:\Users\SUNJIN\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
vectordb = Chroma.from_documents(documents=chunk, embedding=embeddings)

: 